In [1]:
import random
from classes.minion import Minion
from classes.hero import Hero
from scenarios.deck_1 import deck  # Example deck import
from solver.slv import run_single_turn
from state.transition_state import start_turn, apply_results, end_turn

In [2]:
from classes.minion import Minion

# Friendly Board (4 minions)
friendly_minions = [
    Minion(
        name="FriendlyMinion A",
        minion_class="Neutral",
        keywords=[],
        attack=2,
        health=3,
        strat_value=1,
        mana_cost=2
    ),
    Minion(
        name="FriendlyMinion B",
        minion_class="Neutral",
        keywords=[],
        attack=1,
        health=2,
        strat_value=1,
        mana_cost=2
    ),
    Minion(
        name="FriendlyMinion C",
        minion_class="Neutral",
        keywords=[],
        attack=1,
        health=4,
        strat_value=1,
        mana_cost=3
    ),
    Minion(
        name="FriendlyMinion D",
        minion_class="Neutral",
        keywords=[],
        attack=4,
        health=2,
        strat_value=1,
        mana_cost=1
    )
]

# Enemy Board (4 minions)
enemy_minions = [
    Minion(
        name="EnemyMinion #1",
        minion_class="Neutral",
        keywords=[],
        attack=2,
        health=2,
        strat_value=1,
        mana_cost=1
    ),
    Minion(
        name="EnemyMinion #2",
        minion_class="Neutral",
        keywords=[],
        attack=12,
        health=3,
        strat_value=1,
        mana_cost=3
    ),
    Minion(
        name="EnemyMinion #3",
        minion_class="Neutral",
        keywords=[],
        attack=1,
        health=1,
        strat_value=1,
        mana_cost=1
    ),
    Minion(
        name="EnemyMinion #4",
        minion_class="Neutral",
        keywords=[],
        attack=2,
        health=4,
        strat_value=1,
        mana_cost=2
    )
]

# Hand is empty
hand_list = []

In [5]:
def main():
    # 1) Create Hero objects (assuming a Hero class exists)
    hero1 = Hero("Paladin", 30)
    hero2 = Hero("Warrior", 30)

    # 2) Print the deck size before drawing
    print(f"Deck has {len(deck)} cards at the start.")

    # 3) Start turn: hero1 draws exactly one card from the deck
    start_turn(hero1, deck, hand_list)

    # Check what got drawn
    print(f"After draw, hand_list has {len(hand_list)} cards.")
    for k, card in enumerate(hand_list):
        print(f"  Hand card {k}: {card.name} "
              f"(Atk={card.attack}, HP={card.health}, Cost={card.mana_cost})")

    # 4) Build arrays for the solver
    m = len(friendly_minions)   # number of friendly minions on board
    n = len(enemy_minions)      # number of enemy minions on board
    h = len(hand_list)          # number of cards in hand
    M = 5                       # Mana for this turn
    H_hero = 20                 # Opponent hero health

    # ---- Print the counts and minion details ----
    print(f"Number of friendly minions on board: {m}")
    for i, fmn in enumerate(friendly_minions):
        print(f"  - Friendly Minion {i}: {fmn.name} "
              f"(Atk={fmn.attack}, HP={fmn.health})")

    print(f"Number of enemy minions on board: {n}")
    for j, emn in enumerate(enemy_minions):
        print(f"  - Enemy Minion {j}: {emn.name} "
              f"(Atk={emn.attack}, HP={emn.health})")

    print(f"Number of cards in hand: {h}")
    for k, card in enumerate(hand_list):
        print(f"  - Hand Card {k}: {card.name} "
              f"(Atk={card.attack}, HP={card.health}, Cost={card.mana_cost})")

    # Combine board minions + hand minions for indexing in the solver
    combined = friendly_minions + hand_list
    A = [mn.attack for mn in combined]   # attack values
    B = [mn.health for mn in combined]   # health values
    P = [enm.attack for enm in enemy_minions]
    Q = [enm.health for enm in enemy_minions]

    C = [mn.mana_cost for mn in hand_list]
    S = [mn.strat_value for mn in hand_list]

    # 5) Define or load your weights
    weights = {
        "W1": 12,  # kill hero reward
        "W2": 10,  # full board clear
        "W3": 5,   # face damage
        "W4": 1000,  # *really* penalize leaving big minions alive
        "W5": 1,   # less value for friendly minion survival
        "W6": 12,  # reward for minion->minion damage
        "W7": 1,   # reduce penalty for taking damage
        "W8": 4
    }




    # 6) Solve (call the new run_single_turn with a weights dict)
    solution = run_single_turn(
        m=m, n=n, h=h,
        M=M, H_hero=H_hero,
        A=A, B=B, P=P, Q=Q, C=C, S=S,
        weights=weights
    )

    # 7) Apply results to scenario_data
    scenario_data = {
        "A": A,
        "B": B,
        "P": P,
        "Q": Q,
        "H_hero": H_hero,
    }
    apply_results(solution, scenario_data)

    # 8) End turn
    end_turn(hero1, hero2)
    print("Turn complete. Inspect logs above for solver outcome.")


if __name__ == "__main__":
    main()

Deck has 3 cards at the start.
Paladin draws: Minion: Bloodfen Raptor | Class: Neutral | Attack: 3 | Health: 2 | Keywords: 
After draw, hand_list has 3 cards.
  Hand card 0: Murloc Raider (Atk=2, HP=1, Cost=1)
  Hand card 1: Chillwind Yeti (Atk=4, HP=5, Cost=4)
  Hand card 2: Bloodfen Raptor (Atk=3, HP=2, Cost=2)
Number of friendly minions on board: 4
  - Friendly Minion 0: FriendlyMinion A (Atk=2, HP=3)
  - Friendly Minion 1: FriendlyMinion B (Atk=1, HP=2)
  - Friendly Minion 2: FriendlyMinion C (Atk=1, HP=4)
  - Friendly Minion 3: FriendlyMinion D (Atk=4, HP=2)
Number of enemy minions on board: 4
  - Enemy Minion 0: EnemyMinion #1 (Atk=2, HP=2)
  - Enemy Minion 1: EnemyMinion #2 (Atk=12, HP=3)
  - Enemy Minion 2: EnemyMinion #3 (Atk=1, HP=1)
  - Enemy Minion 3: EnemyMinion #4 (Atk=2, HP=4)
Number of cards in hand: 3
  - Hand Card 0: Murloc Raider (Atk=2, HP=1, Cost=1)
  - Hand Card 1: Chillwind Yeti (Atk=4, HP=5, Cost=4)
  - Hand Card 2: Bloodfen Raptor (Atk=3, HP=2, Cost=2)
Gurobi O